In [59]:
import cv2
import numpy as np
cap = cv2.VideoCapture(0)

def nothing(x):
    pass
# Creating a window for later use
cv2.namedWindow('Result')
cv2.namedWindow('Parameter1')
cv2.namedWindow('Parameter2')
cv2.namedWindow('Candidate')
cv2.namedWindow('Target')
# Starting with 100's to prevent error while masking
h_Max,h_Min,s_Max,s_Min,v_Max,v_Min = 179,0,255,0,255,0
# Creating track bar

#For contour finding
cv2.createTrackbar('h_Max', 'Parameter1',0,179,nothing)
cv2.createTrackbar('h_Min', 'Parameter1',0,179,nothing)
cv2.createTrackbar('s_Max', 'Parameter1',0,255,nothing)
cv2.createTrackbar('s_Min', 'Parameter1',0,255,nothing)
cv2.createTrackbar('v_Max', 'Parameter1',0,255,nothing)
cv2.createTrackbar('v_Min', 'Parameter1',0,255,nothing)
cv2.createTrackbar('Threshold', 'Parameter1',0,1,nothing)
cv2.createTrackbar('Blur', 'Parameter1',0,1,nothing)
cv2.createTrackbar('Dilate', 'Parameter1',0,1,nothing)
cv2.createTrackbar('Open', 'Parameter1',0,1,nothing)
cv2.createTrackbar('Close', 'Parameter1',0,1,nothing)
cv2.createTrackbar('Erode', 'Parameter1',0,1,nothing)
cv2.createTrackbar('FindCountours', 'Parameter1',0,1,nothing)
cv2.createTrackbar('Area_Low', 'Parameter1',0,6000,nothing)

#For cropped Image
cv2.createTrackbar('h_Max', 'Parameter2',0,179,nothing)
cv2.createTrackbar('h_Min', 'Parameter2',0,179,nothing)
cv2.createTrackbar('s_Max', 'Parameter2',0,255,nothing)
cv2.createTrackbar('s_Min', 'Parameter2',0,255,nothing)
cv2.createTrackbar('v_Max', 'Parameter2',0,255,nothing)
cv2.createTrackbar('v_Min', 'Parameter2',0,255,nothing)
cv2.createTrackbar('Threshold', 'Parameter2',0,1,nothing)
cv2.createTrackbar('Blur', 'Parameter2',0,1,nothing)
cv2.createTrackbar('Dilate', 'Parameter2',0,1,nothing)
cv2.createTrackbar('Open', 'Parameter2',0,1,nothing)
cv2.createTrackbar('Close', 'Parameter2',0,1,nothing)
cv2.createTrackbar('Erode', 'Parameter2',0,1,nothing)
cv2.createTrackbar('Number', 'Parameter2',0,9,nothing)

#Neural Network Parameter : Training Accuracy:1 Testing Accuracy: 0.8
W = np.load('W.npy')
b = np.load('b.npy') 
dim = (28,28)
def getTrackbarPos(switch):
    if switch==1:
        h_Max = cv2.getTrackbarPos('h_Max','Parameter1')
        s_Max = cv2.getTrackbarPos('s_Max','Parameter1')
        v_Max = cv2.getTrackbarPos('v_Max','Parameter1')
        h_Min = cv2.getTrackbarPos('h_Min','Parameter1')
        s_Min = cv2.getTrackbarPos('s_Min','Parameter1')
        v_Min = cv2.getTrackbarPos('v_Min','Parameter1')
        Threshold = cv2.getTrackbarPos('Threshold','Parameter1')
        Blur = cv2.getTrackbarPos('Blur','Parameter1')
        Dilate = cv2.getTrackbarPos('Dilate','Parameter1')
        Open = cv2.getTrackbarPos('Open','Parameter1')
        Close = cv2.getTrackbarPos('Close','Parameter1')
        Erode = cv2.getTrackbarPos('Erode','Parameter1')
        FindCountours = cv2.getTrackbarPos('FindCountours','Parameter1')
        Area_Low = cv2.getTrackbarPos('Area_Low','Parameter1')
        return np.array([h_Max,h_Min,s_Max,s_Min,v_Max,v_Min,Threshold,Blur,Dilate,Open,Close,Erode,FindCountours,Area_Low])
    if switch==2:
        h_Max = cv2.getTrackbarPos('h_Max','Parameter2')
        s_Max = cv2.getTrackbarPos('s_Max','Parameter2')
        v_Max = cv2.getTrackbarPos('v_Max','Parameter2')
        h_Min = cv2.getTrackbarPos('h_Min','Parameter2')
        s_Min = cv2.getTrackbarPos('s_Min','Parameter2')
        v_Min = cv2.getTrackbarPos('v_Min','Parameter2')
        Threshold = cv2.getTrackbarPos('Threshold','Parameter2')
        Blur = cv2.getTrackbarPos('Blur','Parameter2')
        Dilate = cv2.getTrackbarPos('Dilate','Parameter2')
        Open = cv2.getTrackbarPos('Open','Parameter2')
        Close = cv2.getTrackbarPos('Close','Parameter2')
        Erode = cv2.getTrackbarPos('Erode','Parameter2')
        return np.array([h_Max,h_Min,s_Max,s_Min,v_Max,v_Min,Threshold,Blur,Dilate,Open,Close,Erode])
    else:
        return 0
    # get info from track bar and appy to result
def bound(switch):
    parameter = getTrackbarPos(switch)
    h_Max,h_Min,s_Max,s_Min,v_Max,v_Min= parameter[0],parameter[1],parameter[2],parameter[3],parameter[4],parameter[5]
    lower_blue = np.array([h_Min,s_Min,v_Min])
    upper_blue = np.array([h_Max,s_Max,v_Max])
    return lower_blue,upper_blue
def GetMask(frame,switch):
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    lower_blue,upper_blue = bound(switch)
    mask = cv2.inRange(hsv,lower_blue, upper_blue)
    return mask
def threshold(img,switch):
    Threshold = getTrackbarPos(switch)[6]
    if Threshold == 1: #Switch
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        Tmask = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
        cv2.THRESH_BINARY,11,2)
        img = cv2.bitwise_and(img,img,mask = Tmask)
        return img
    else:
        return img
def Blur(img,switch):
    Blur = getTrackbarPos(switch)[7]
    if Blur == 1: #Switch
        img = cv2.medianBlur(img,5)
        return img
    else:
        return img
def Dilate(img,switch):
    Dilate = getTrackbarPos(switch)[8]
    if Dilate == 1:#Switch
        kernel = np.ones((10,10),np.uint8)
        img = cv2.dilate(img,kernel,iterations = 1)
        return img
    else:
        return img
def Open(img,switch):
    Open = getTrackbarPos(switch)[9]
    if Open == 1:#Switch
        kernel = np.ones((10,10),np.uint8)
        img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
        return img
    else:
        return img
def Erode(img,switch):
    Erode = getTrackbarPos(switch)[10]
    if Erode == 1:#Switch
        kernel = np.ones((5,5),np.uint8)
        img = cv2.erode(img,kernel,iterations = 1)
        return img
    else:
        return img
def Close(img,switch):
    Close = getTrackbarPos(switch)[11]
    if Close == 1:#Switch
        kernel = np.ones((10,10),np.uint8)
        img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
        return img
    else:
        return img
def Filter(frame,switch):
    img = cv2.bitwise_and(frame,frame,mask = GetMask(frame,switch)) #HSL Filtering
    img = Blur(img,switch) #Blur image
    img = threshold(img,switch) #Adaptive Threshold
    #Morphological Transformations
    img = Close(img,switch)
    img = Dilate(img,switch)
    img = Erode(img,switch)
    img = Open(img,switch) 
    return img
def countour(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _,contours,_ = cv2.findContours(gray,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    return contours

def neuralprocess(W,b,Input,dim):
    Input = cv2.resize(Input,dim, interpolation = cv2.INTER_AREA)
    Input = Input.reshape((1,dim[0]*dim[1]))
    output = np.maximum((np.matmul(Input, W)+b),0)
    #print np.argmax(output)
    return np.argmax(output)
while(1):
    _, frame = cap.read()
    # Normal masking algorithm
    result = Filter(frame,1)
    #Label Contours
    if cv2.getTrackbarPos('FindCountours','Parameter1')==1:
        for cnt in countour(result): 
            x,y,w,h = cv2.boundingRect(cnt)
            Cand = Filter(frame[y:y+h,x:x+w],2)
            
            kernel = np.ones((5,5),np.uint8)
            Cand = cv2.cvtColor(Cand, cv2.COLOR_BGR2GRAY)
            if cv2.contourArea(cnt)>=cv2.getTrackbarPos('Area_Low','Parameter1'):
                if neuralprocess(W,b,Cand,dim) == cv2.getTrackbarPos('Number','Parameter2'): #Represents yes
                    cv2.imshow('Target',Cand)
                    cv2.rectangle(result,(x,y),(x+w,y+h),(255,255,255),2)
                else:
                    cv2.rectangle(result,(x,y),(x+w,y+h),(0,0,255),2)
                    cv2.imshow('Candidate',Cand)
    cv2.imshow('Result',result)
    cv2.imshow('Original',frame)
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break

cap.release()

cv2.destroyAllWindows()


KeyboardInterrupt: 

In [ ]:
#High error prob.

In [60]:
cap.release()
cv2.destroyAllWindows()

For HSV, Hue range is [0,179], Saturation range is [0,255] and Value range is [0,255]
More Labeling : http://docs.opencv.org/3.1.0/dd/d49/tutorial_py_contour_features.html

For Target Recognition
Parameter1:
h_max 150
h_min 116
s_max 190
s_min 54
v_max 255
v_min 108
Threshold 0
Blur 1
Dilate 1
Open 0
Close 0
Erode 0
Number: 0
Parameter2:
h_max 179
h_min 115
s_max 255
s_min 0
v_max 255
v_min 0
Threshold 0
Blur 0
Dilate 0
Open 0
Close 0
Erode 0

Image Source:
https://img.clipartfest.com/f1ff878c07c595f22322c880901cd7bf_number-0-clip-art-number-0-clipart_600-843.png